In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")

In [4]:
os.environ["TAVILY_API_KEY"]= TAVILY_API_KEY
os.environ["LANGCHAIN_API_KEY"]= LANGCHAIN_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["SERPER_API_KEY"]= SERPER_API_KEY
os.environ["GOOGLE_API_KEY"]= GOOGLE_API_KEY
os.environ["LANGSMITH_TRACING"]= "true"
os.environ["LANGSMITH_ENDPOINT"]= "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"]= LANGSMITH_PROJECT

In [5]:
LANGSMITH_PROJECT

'pre-req'

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

d:\langgraph\lang-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
while True:
    question = input("Ask your question: ")
    if question!="quit":
        print(llm.invoke(question).content)
    else:
        print("Have a great day. Bye!")
        break

Have a great day. Bye!


## Integrating memory

In [8]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [9]:
store ={}

In [10]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [11]:
config = {"configurable": {"session_id":"first_chat"}}

In [12]:
model_with_memory = RunnableWithMessageHistory(llm,get_session_history)

In [13]:
model_with_memory.invoke(("Hello. I am Shochcho"), config=config).content

"Hello Shochcho!  It's nice to meet you.  How can I help you today?"

In [14]:
model_with_memory.invoke(("What is my name?"), config=config).content

'Your name is Shochcho.'

In [15]:
store

{'first_chat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello. I am Shochcho', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Shochcho!  It's nice to meet you.  How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-dbc367a3-4821-43f8-9373-733aa3ee2c5d-0', usage_metadata={'input_tokens': 8, 'output_tokens': 23, 'total_tokens': 31, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Shochcho.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-25168804-1af4-42df-81fc-1b3f7b5b643f-0', usage_metadata={'input_tokens': 37, 'output_tokens': 8, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}})])

## RAG

In [16]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Step 1: Reading the txt files from source directory
loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

# Step 2: Creating Chunks using RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=30,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

# Step 3: Creating Retriever using FAISS Vector DB

# Create the FAISS database from the document embeddings
db = FAISS.from_documents(new_docs, embeddings)

# Create a retriever from the FAISS database
retriever = db.as_retriever(search_kwargs={"k": 4})

# Example usage of retriever
query = "What is the content of the document?"
results = retriever.get_relevant_documents(query)
for idx, result in enumerate(results, 1):
    print(f"Result {idx}:\n{result.page_content}\n")